In [20]:
import re
coords = re.findall("\(.+\)", "1110 Boston Road\nBronx, NY 10456\n(40.8276026690005, -73.90447525699966)")
type(coords[0])

## Funcion para extraer una cadena entre parentesis.

#import re
#def find_lat(loc):
#    coords = re.findall("\(.+\)", "1110 Boston Road\nBronx, NY 10456\n(40.8276026690005, -73.90447525699966)")
#    lat = coords[0].split(",")[0].replace("(", "")
#    return lat
    
#data["hs_directory"]["lat"]=data["hs_directory"]["Location 1"].apply(find_lat)

#print(data["hs_directory"]["lat"].head())

str

In [21]:
coords = re.findall("\(.+\)", "1110 Boston Road\nBronx, NY 10456\n(40.8276026690005, -73.90447525699966)")
long = coords[0].split(",")[1].replace("(", "")
long

' -73.90447525699966)'

In [22]:
long.replace(")", "")

' -73.90447525699966'

## Extraction of strings, convirting columns to numeric data, creating new columns

In [23]:

import re
def find_long(loc):
    coords = re.findall("\(.+\)",loc") ## Selecting just inside parenthesis
    long = coords[0].split(",")[1].replace(")", "").strip()
    return long

data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(find_long)

data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"], errors="coerce")

data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"],errors="coerce")

print(data["hs_directory"].head())


SyntaxError: EOL while scanning string literal (<ipython-input-23-d8fb629bd037>, line 4)

In [ ]:
import pandas as pd
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

data = {}
for files in data_files:
    df = pd.read_csv("schools/{0}".format(files))
    dfname = files.replace(".csv","")
    data[dfname] = df


In [ ]:
all_survey=pd.read_csv("schools/survey_all.txt",delimiter="\t",encoding="windows-1252")

d75_survey=pd.read_csv("schools/survey_d75.txt",delimiter="\t",encoding="windows-1252")

survey=pd.concat([all_survey, d75_survey], axis=0)
print(survey.head(5))


In [28]:
survey["DBN"] = survey["dbn"]

survey_fields = [
    "DBN", 
    "rr_s", 
    "rr_t", 
    "rr_p", 
    "N_s", 
    "N_t", 
    "N_p", 
    "saf_p_11", 
    "com_p_11", 
    "eng_p_11", 
    "aca_p_11", 
    "saf_t_11", 
    "com_t_11", 
    "eng_t_11", 
    "aca_t_11", 
    "saf_s_11", 
    "com_s_11", 
    "eng_s_11", 
    "aca_s_11", 
    "saf_tot_11", 
    "com_tot_11", 
    "eng_tot_11", 
    "aca_tot_11",
]
survey = survey.loc[:,survey_fields]
data["survey"] = survey

print(survey.head())

      DBN  rr_s  rr_t  rr_p    N_s   N_t    N_p  saf_p_11  com_p_11  eng_p_11  \
0  01M015   NaN    88    60    NaN  22.0   90.0       8.5       7.6       7.5   
1  01M019   NaN   100    60    NaN  34.0  161.0       8.4       7.6       7.6   
2  01M020   NaN    88    73    NaN  42.0  367.0       8.9       8.3       8.3   
3  01M034  89.0    73    50  145.0  29.0  151.0       8.8       8.2       8.0   
4  01M063   NaN   100    60    NaN  23.0   90.0       8.7       7.9       8.1   

      ...      eng_t_11  aca_t_11  saf_s_11  com_s_11  eng_s_11  aca_s_11  \
0     ...           7.6       7.9       NaN       NaN       NaN       NaN   
1     ...           8.9       9.1       NaN       NaN       NaN       NaN   
2     ...           6.8       7.5       NaN       NaN       NaN       NaN   
3     ...           6.8       7.8       6.2       5.9       6.5       7.4   
4     ...           7.8       8.1       NaN       NaN       NaN       NaN   

   saf_tot_11  com_tot_11  eng_tot_11  aca_tot_11 

In [ ]:
data["hs_directory"]["DBN"] = data["hs_directory"]["dbn"]

def pad_csd(num):
    string_representation = str(num)
    if len(string_representation) > 1:
        return string_representation
    else:
        return string_representation.zfill(2)
    
data["class_size"]["padded_csd"] = data["class_size"]["CSD"].apply(pad_csd)
data["class_size"]["DBN"] = data["class_size"]["padded_csd"] + data["class_size"]["SCHOOL CODE"]
print(data["class_size"].head())

In [ ]:
class_size=data["class_size"]
class_size=class_size[class_size["GRADE "] == "09-12"]
class_size=class_size[class_size["PROGRAM TYPE"] == "GEN ED"]
print(class_size.head())

## Computing Average Class Sizes

- Find the average values for each column associated with each DBN in class_size.
    - Use the pandas.DataFrame.groupby() method to group class_size by DBN.
    - Use the agg() method on the resulting pandas.core.groupby object, along with the numpy.mean() function as an argument, to calculate the average of each group.
    -Assign the result back to class_size.
- Reset the index to make DBN a column again.
     - Use the pandas.DataFrame.reset_index() method, along with the keyword argument inplace=True.
- Assign class_size back to the class_size key of the data dictionary.
- Display the first few rows of data["class_size"] to verify that everything went okay.

In [ ]:
import numpy
newclass_size=class_size.groupby("DBN").agg(numpy.mean).reset_index()
newclass_size.reset_index(inplace=True)
data["class_size"] = newclass_size
print(data["class_size"].head()) # Se realiza la media de cada una de las columnas agrupadas por grupo DBN.


## Condensing the Demographics Data Set

- Filter demographics, only selecting rows in data["demographics"] where schoolyear is 20112012.
    - schoolyear is actually an integer, so be careful about how you perform your comparison.
- Display the first few rows of data["demographics"] to verify that the filtering worked.

In [ ]:
demo=data["demographics"]
demo=demo[demo["schoolyear"] == 20112012 ]
data["demographics"] = demo
data["demographics"].head()

## Condensing the Graduation Data Set

- Filter graduation, only selecting rows where the Cohort column equals 2006.
- Filter graduation, only selecting rows where the Demographic column equals Total Cohort.
- Display the first few rows of data["graduation"] to verify that everything worked properly.

In [ ]:
demo1=data["graduation"]
demo1=demo1[(demo1["Cohort"] == "2006") & (demo1["Demographic"] == "Total Cohort")]
data["graduation"] = demo1



## Converting AP Test Scores 
- Convert each of the following columns in ap_2010 to numeric values using the pandas.to_numeric() function with the keyword argument errors="coerce".
    - AP Test Takers
    - Total Exams Taken
    - Number of Exams with scores 3 4 or 5
- Display the column types using the dtypes attribute.


In [ ]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']

for i in cols:
    s = data["ap_2010"][i]
    data["ap_2010"][i]=pd.to_numeric(s, errors="coerce")
data.dtypes
                    

## Performing the Inner Joins


In [29]:
to_merge = ["class_size", "demographics", "survey", "hs_directory"]

for m in to_merge:
    combined = combined.merge(data[m], on="DBN", how="inner")

print(combined.head(5))
print(combined.shape)


      DBN                                        SCHOOL NAME  \
0  01M292      HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448                UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                         EAST SIDE COMMUNITY SCHOOL   
3  01M509                            MARTA VALLE HIGH SCHOOL   
4  01M539  NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...   

  Num of SAT Test Takers SAT Critical Reading Avg. Score SAT Math Avg. Score  \
0                     29                             355                 404   
1                     91                             383                 423   
2                     70                             377                 402   
3                     44                             390                 433   
4                    159                             522                 574   

  SAT Writing Avg. Score                      SchoolName  AP Test Takers   \
0                    363                             NaN 

## Performing the Left Joins

- Use the pandas pandas.DataFrame.merge() method to merge the ap_2010 data set into combined.
    - Make sure to specify how="left" as a keyword argument to indicate the correct join type.
    - Make sure to assign the result of the merge operation back to combined.
- Use the pandas df.merge() method to merge the graduation data set into combined.
    - Make sure to specify how="left" as a keyword argument to get the correct join type.
    - Make sure to assign the result of the merge operation back to combined.
- Display the first few rows of combined to verify that the correct operations occurred.
- Use the pandas.DataFrame.shape attribute to display the shape of the dataframe and see how many rows now exist.

In [30]:
combined = data["sat_results"]
combined = combined.merge(data["ap_2010"], on="DBN", how="left")
combined = combined.merge(data["graduation"], on="DBN", how="left")
print(combined.head(5))
print(combined.shape)




      DBN                                    SCHOOL NAME  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

  Num of SAT Test Takers SAT Critical Reading Avg. Score SAT Math Avg. Score  \
0                     29                             355                 404   
1                     91                             383                 423   
2                     70                             377                 402   
3                      7                             414                 401   
4                     44                             390                 433   

  SAT Writing Avg. Score                    SchoolName  AP Test Takers   \
0                    363                           NaN              NaN   
1       

## Performing the Inner Joins

- Merge class_size into combined. Then, merge demographics, survey, and hs_directory into combined one by one, in that order.
    - Be sure to follow the exact order above.
    - Remember to specify the correct column to join on, as well as the correct join type.
- Display the first few rows of combined to verify that the correct operations occurred.
- Call pandas.DataFrame.shape() to display the shape of the dataframe to see how many rows now exist.

In [31]:
to_merge = ["class_size", "demographics", "survey", "hs_directory"]

for m in to_merge:
    combined = combined.merge(data[m], on="DBN", how="inner")

print(combined.head(5))
print(combined.shape)

      DBN                                        SCHOOL NAME  \
0  01M292      HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448                UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                         EAST SIDE COMMUNITY SCHOOL   
3  01M509                            MARTA VALLE HIGH SCHOOL   
4  01M539  NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...   

  Num of SAT Test Takers SAT Critical Reading Avg. Score SAT Math Avg. Score  \
0                     29                             355                 404   
1                     91                             383                 423   
2                     70                             377                 402   
3                     44                             390                 433   
4                    159                             522                 574   

  SAT Writing Avg. Score                      SchoolName  AP Test Takers   \
0                    363                             NaN 

## Filling in missing values

- Calculate the means of all of the columns in combined using the pandas.DataFrame.mean() method.
- Fill in any missing values in combined with the means of the respective columns using the pandas.DataFrame.fillna() method.
- Fill in any remaining missing values in combined with 0 using the df.fillna() method.
- Display the first few rows of combined to verify that the correct operations occurred.

In [32]:
mean=combined.mean()
combined=combined.fillna(mean)
combined=combined.fillna(0)

## Adding a School District Column for Mapping

- Write a function that extracts the first two characters of a string and returns them.
- Apply the function to the DBN column of combined, and assign the result to the school_dist column of combined.
- Display the first few items in the school_dist column of combined to verify the results.

In [33]:
def character(val):
    ext=val[0:2]
    return ext
combined["school_dist"]=combined["DBN"].apply(character)
combined["school_dist"].head()

0    01
1    01
2    01
3    01
4    01
Name: school_dist, dtype: object

## Data Cleaning Analyzing and Visualizing the Data


# Finding Correlations With the r Value

- Use the pandas.DataFrame.corr() method on the combined dataframe to find all possible correlations. Assign the result to correlations.
- Filter correlations so that it only shows correlations for the column sat_score.
- Display all of the rows in correlations and look them over.

In [42]:
correlations = combined.corr()


In [44]:
combined["sat_score"]

KeyError: 'sat_score'

##  Mapping the Schools With Basemap

In [ ]:
from mpl_toolkits.basemap import Basemap
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='i'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes=combined["lon"].tolist()
latitudes = combined["lat"].tolist()
m.scatter(longitudes, latitudes, s=20, zorder=2, latlon=True)
plt.show()


## Plotting our statistics


In [ ]:
from mpl_toolkits.basemap import Basemap

m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='i'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = combined["lon"].tolist()
latitudes = combined["lat"].tolist()
m.scatter(longitudes, latitudes, s=20, zorder=2, latlon=True, c=combined["ell_percent"], cmap="summer")
plt.show()

## Plotting percentage by districts

In [ ]:
from mpl_toolkits.basemap import Basemap

m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='i'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = districts["lon"].tolist()
latitudes = districts["lat"].tolist()
m.scatter(longitudes, latitudes, s=50, zorder=2, latlon=True, c=districts["ell_percent"], cmap="summer")
plt.show()